In [1]:
import numpy as np
import pandas as pd

from tqdm import tqdm #显示运行进度条

# # 画图库
# %matplotlib inline
# %config InlineBackend.figure_format = 'retina'
# import seaborn as sb
# import matplotlib.pyplot as plt

# 文字处理库
from keras.preprocessing.text import Tokenizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer 
# nltk.download('stopwords') #下载停止词
import string
from keras.preprocessing import sequence
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
labels = train_data.columns[2:].tolist()
tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')
all_comments = pd.concat([train_data['comment_text'], test_data['comment_text']])


Using TensorFlow backend.


In [2]:
stop_words = set(stopwords.words('english'))
stop_words.update(string.punctuation)
comments_refined = []
for line in tqdm(all_comments):
    comments_refined.append(' '.join([word for word in tokenizer.tokenize(line) if word not in stop_words]))

100%|██████████| 312735/312735 [00:10<00:00, 30651.35it/s]


In [4]:

vectorizer = TfidfVectorizer(
    strip_accents='unicode',
    analyzer='word',
#     ngram_range=(1, 1),
    use_idf=True,
    smooth_idf=True,
    sublinear_tf=True,
    max_features=10000).fit(comments_refined)
train_features, test_features = map(lambda x: vectorizer.transform(x), [train_data['comment_text'], test_data['comment_text']] )
scores = []
submission = pd.DataFrame.from_dict({'id': test_data['id']})
for label in tqdm(labels):
    train_target = train_data[label]
    classifier = LogisticRegression(C=1, 
                                    solver='sag'
#                                     class_weight='balanced'
                                   )
    cv_score = np.mean(cross_val_score(classifier, 
                                       train_features, 
                                       train_data[label], 
                                       cv=3, 
                                       scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(label, cv_score))

    classifier.fit(train_features, train_target)
    submission[label] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))
submission.to_csv('submission_simple.csv', index=False)

  0%|          | 0/6 [00:00<?, ?it/s]

CV score for class toxic is 0.9689433208062592


 17%|█▋        | 1/6 [00:09<00:45,  9.18s/it]

CV score for class severe_toxic is 0.9828255058132512


 33%|███▎      | 2/6 [00:18<00:36,  9.22s/it]

...

CV score for class insult is 0.9754275751914161


 83%|████████▎ | 5/6 [00:45<00:09,  9.10s/it]

CV score for class identity_hate is 0.9704158533174798


100%|██████████| 6/6 [00:53<00:00,  8.95s/it]


Total CV score is 0.97689791100767
